# Composition → Microstructure Sensitivity Pipeline

This notebook reproduces the original Python script in a Jupyter‑friendly, step‑by‑step format.

* **Goal**: Predict γ′ phase fraction from alloy composition and analyse element sensitivities.
* **Method**: Polynomial Chaos Expansion (PCE) surrogate model with Sobol sensitivity indices.
* **Data**: An HDF5 file containing element mass fractions and measured phase fractions.

Feel free to execute cells sequentially or modify parameters (e.g., file path, polynomial order range).

In [ ]:

#------------------------------------------------------------------------------#
# 1. Imports & environment setup                                               #
#------------------------------------------------------------------------------#

import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Surrogate modelling
import chaospy as cp
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Plotting defaults for notebook
%matplotlib inline
plt.rcParams.update({'font.size': 14, 'font.weight': 'bold'})
sns.set_style('whitegrid')


## 2. Load the dataset from HDF5

Change `hdf5_path` if your data file has a different name or location. The file is expected to contain three datasets:

* **`composition`** — 2‑D array of mass fractions (Ni, Cr, Nb, Mo, Ti, Al, Co)
* **`g_prime`** — γ′ phase fraction (float per alloy)
* **`g_dprime`** — γ′′ phase fraction (float per alloy)  

The loader converts these to NumPy arrays for further processing.

In [ ]:

#------------------------------------------------------------------------------#
# 2. Data ingestion                                                            #
#------------------------------------------------------------------------------#

hdf5_path = "Insert_your_file_name.hdf5"   # <-- update if needed

with h5py.File(hdf5_path, 'r') as f:
    composition_mass = np.array(f['composition'], dtype=np.float32)
    g_prime = np.array(f['g_prime'], dtype=np.float32)
    g_dprime = np.array(f['g_dprime'], dtype=np.float32)

print("Loaded:", composition_mass.shape[0], "alloys")


## 3. Convert mass fractions → mole fractions & build a DataFrame

Conversion uses atomic weights and assumes iron (Fe) is the balance element.  Values are expressed as *mole %*.

In [ ]:

#------------------------------------------------------------------------------#
# 3. Mass-→mole conversion                                                     #
#------------------------------------------------------------------------------#

atomic_weights = {
    'Fe': 55.845, 'Ni': 58.69, 'Cr': 52.00, 'Nb': 92.90,
    'Mo': 95.95, 'Ti': 47.867, 'Al': 26.98154, 'Co': 58.933
}

Ni, Cr, Nb, Mo, Ti, Al, Co = ([] for _ in range(7))

for row in composition_mass:
    Fe_mass = 1.0 - row.sum()        # balancing Fe mass fraction
    m_total = (
        Fe_mass / atomic_weights['Fe']
        + row[0] / atomic_weights['Ni']
        + row[1] / atomic_weights['Cr']
        + row[2] / atomic_weights['Nb']
        + row[3] / atomic_weights['Mo']
        + row[4] / atomic_weights['Ti']
        + row[5] / atomic_weights['Al']
        + row[6] / atomic_weights['Co']
    )

    Ni.append(row[0] / atomic_weights['Ni'] / m_total * 100)
    Cr.append(row[1] / atomic_weights['Cr'] / m_total * 100)
    Nb.append(row[2] / atomic_weights['Nb'] / m_total * 100)
    Mo.append(row[3] / atomic_weights['Mo'] / m_total * 100)
    Ti.append(row[4] / atomic_weights['Ti'] / m_total * 100)
    Al.append(row[5] / atomic_weights['Al'] / m_total * 100)
    Co.append(row[6] / atomic_weights['Co'] / m_total * 100)

df = pd.DataFrame({
    "Ni (mole %)": Ni, "Cr (mole %)": Cr, "Nb (mole %)": Nb,
    "Al (mole %)": Al, "Ti (mole %)": Ti, "Co (mole %)": Co,
    "Mo (mole %)": Mo, "γ′ (mole %)": g_prime, "γ′′ (mole %)": g_dprime
})

df.head()


## 4. Explore variable distributions

The next cell plots histograms with KDE curves for each composition and phase‑fraction column.

In [ ]:

#------------------------------------------------------------------------------#
# 4. Visualise distributions                                                   #
#------------------------------------------------------------------------------#

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(18, 14))
axes = axes.flatten()

for ax, column in zip(axes, df.columns):
    sns.histplot(df[column], kde=True, ax=ax)
    ax.set_xlabel(column)
    ax.set_ylabel("Count")

plt.tight_layout()
plt.show()


## 5. Prepare inputs & target for PCE surrogate

* **Features (X)**: first seven columns (element mole %).
* **Target (y)**: γ′ mole %.

Uniform distributions are defined across the observed range of each feature and combined into an independent joint distribution.

In [ ]:

#------------------------------------------------------------------------------#
# 5. Feature/target arrays + distributions                                     #
#------------------------------------------------------------------------------#

X = df.iloc[:, 0:7].values.astype('float64')
y = df['γ′ (mole %)'].values.astype('float64')

dists = [
    cp.Uniform(df[col].min(), df[col].max())
    for col in df.columns[:7]
]
joint_dist = cp.J(*dists)

# Train/val/test split: 80/10/10
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.10, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1111, random_state=42
)
print("Train:", X_train.shape, " Val:", X_val.shape, " Test:", X_test.shape)


## 6. Hyper‑parameter search for polynomial order

Loop over candidate orders (1 → 4), fit a PCE on the training set, predict on the validation set, and store metrics.

In [ ]:

#------------------------------------------------------------------------------#
# 6. Polynomial order sweep                                                    #
#------------------------------------------------------------------------------#

orders = range(1, 5)
val_mse, val_r2 = [], []

for order in orders:
    poly = cp.expansion.stieltjes(order, joint_dist)
    model = cp.fit_regression(poly, X_train.T, y_train)
    y_pred = model(*X_val.T)
    mse = mean_squared_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    val_mse.append(mse)
    val_r2.append(r2)
    print(f"Order {order}: val MSE={mse:.4f}, R²={r2:.4f}")

best_order = orders[int(np.argmin(val_mse))]
print("\nSelected polynomial order:", best_order)


## 7. Train final PCE & evaluate on hold‑out test set

In [ ]:

#------------------------------------------------------------------------------#
# 7. Final model fit & test evaluation                                         #
#------------------------------------------------------------------------------#

poly_final = cp.expansion.stieltjes(best_order, joint_dist)
model_final = cp.fit_regression(poly_final, X_train.T, y_train)

y_test_pred = model_final(*X_test.T)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)
print(f"Test MSE={test_mse:.4f}, R²={test_r2:.4f}")

# KDE overlay
plt.figure(figsize=(8,5))
sns.kdeplot(y_test_pred, linewidth=3, label="Predicted γ′")
sns.kdeplot(y_test, linewidth=3, label="Actual γ′")
plt.xlabel("γ′ (mole %)")
plt.ylabel("Kernel density")
plt.legend()
plt.show()


## 8. Global sensitivity analysis (Sobol indices)

Chaospy can compute Sobol indices directly from the fitted PCE coefficients.

In [ ]:

#------------------------------------------------------------------------------#
# 8. Sobol sensitivity indices                                                 #
#------------------------------------------------------------------------------#

main_sobol = cp.Sens_m(model_final, joint_dist)
total_sobol = cp.Sens_t(model_final, joint_dist)

elements = ['Ni', 'Cr', 'Nb', 'Al', 'Ti', 'Co', 'Mo']
sobol_df = pd.DataFrame({
    'Element': elements,
    'Main': main_sobol,
    'Total': total_sobol
})

display(sobol_df)

# Bar plot
sobol_df.set_index('Element').plot.bar(figsize=(10,5))
plt.title("Sobol sensitivity indices for γ′")
plt.ylabel("Sobol index")
plt.ylim(0, 1)
plt.show()


## 9. Next steps

* **γ′′ modelling**: Repeat Section 5‑8 with `y = df['γ′′ (mole %)']`.
* **Better distributions**: Replace uniform priors with empirical or kernel‑density estimates.
* **Interaction visualisations**: Explore pairwise elemental effects using partial‑dependence or ALE plots.

---
*Notebook generated on 2025-08-06.*